[Документация](https://docs.djangoproject.com/en/5.2/topics/forms/)

## Form

Класс `Form` в Django предоставляет механизм для создания, проверки и обработки форм на стороне сервера. Он может использоваться для обработки данных, отправленных пользователем через формы HTML, и предоставляет функции валидации и очистки данных.

### Схема работы
1. Определите класс формы, наследуясь от `django.forms.Form`, и определите поля ввода и их свойства.
2. В представлении Django создайте экземпляр формы и передайте его в контекст шаблона.
3. В шаблоне отобразите форму и обработайте отправку данных.
4. В представлении Django обработайте полученные данные, проверьте их валидность и выполните необходимые действия (например, сохранение данных в базе данных).

### Типы запросов
Django обрабатывает два основных типа запросов, связанных с формами:
- `GET`: используется для отображения формы и получения данных от пользователя.
- `POST`: используется для отправки данных формы на сервер для обработки.

### Метод `is_valid()`
Метод `is_valid()` вызывается на экземпляре формы для проверки валидности отправленных данных. Если данные валидны, метод возвращает `True`, иначе - `False`. Ошибки валидации доступны через атрибут `errors` формы.
Если данные формы не проходят валидацию, то `cleaned_data` будет содержать только валидные поля.

### Валидация данных и ошибки
Django автоматически валидирует данные в полях формы на основе их типа и заданных ограничений (например, обязательные поля, максимальная длина и т. д.). Если валидация не проходит, в атрибуте `errors` формы будут содержаться ошибки, связанные с каждым полем.

### Пример кода формы
```python
from django import forms

class ContactForm(forms.Form):
    name = forms.CharField(label='Your name', max_length=100)
    email = forms.EmailField(label='Your email')
    message = forms.CharField(widget=forms.Textarea, label='Your message')

# В представлении
from django.shortcuts import render
from .forms import ContactForm

def contact(request):
    if request.method == 'POST':
        form = ContactForm(request.POST)
        if form.is_valid():
            # Обработка данных формы
            pass
    else:
        form = ContactForm()
    return render(request, 'contact.html', {'form': form})

# В шаблоне
<form method="post">
    {% csrf_token %}
    {{ form.as_p }}
    <button type="submit">Send</button>
</form>
```

### Вывод формы в шаблон

#### Способ 1
```django
 <form action="{% url 'news:add_news' %}" method="POST">
    {{ form.as_p }}
  </form>
```
#### Способ 2
```django
<form action="{% url 'news:add_news' %}" method="POST">
	{{ form.non_field_errors }}
	<div class="mb-3">
	  <label for="{{ form.title.id_for_label }}" class="form-label">{{ form.title.label }}</label>
	  {{ form.title }}
	  <div class="invalid-feedback">{{ form.title.errors }}</div>
	</div>
</form>
```
#### Способ 3
```django
<form action="{% url 'news:add_news' %}" method="POST">
  {% for field in form %}
	<div class="form-group">
	  {{ field.label_tag }}
	  {{ field }}
      <div class="invalid-feedback">{{ form.category.errors }}
    </div>
  {% endfor %}
</form>
```

### Рендеринг формы в шаблоне

```python
<form method="POST" action="add_news">
	{% csrf_token %}
	# Автоматический рендеринг
	{{ form.as_p }}       # Каждая форма в параграфе
	{{ forms.as_ul }}     # Каждая форма в элементе списка
	{{ forms.as_table }}  # Каждая форма в ячейке таблицы
	# Ручной рендеринг формы
	{{ form.non_field.errors }}
	<div class="form-group">
		<label for="{{ form.title.id_for_label }}">Название</label>
			{{ form.title }}
		<div class="valid-feedback">
			{{ form.title.errors }}
		</div>
	</div>
	# Рендеринг в цикле
	{% for field for in %}
		{{ field.label_tag }}
		{{ field }}
	{% endfor %}
	# В список и таблицу нужно оборачивать самостоятельно
	<button type="submit">Отправить</button>
</form>
```

## modelForm
Форма на основе модели

### Процесс создания формы в общем виде:
#### 1.Создаётся (или выбирается существующая) модель.
```python
# library/models.py
from django.db import models
from django.core.validators import MinValueValidator
# Создадим модель, в которой будем хранить данные о книгах
class Book(models.Model):
    name = models.CharField(max_length=200)   # Название
    isbn = models.CharField(max_length=100)   # Индекс издания
    pages = models.IntegerField(              # Количество страниц
        validators=[MinValueValidator(1)]
    )
```

#### 2. Создаётся класс формы (наследуется от ModelForm), в нём указывается, с какой моделью будет работать эта форма.
```python
# library/forms.py
from django import forms
from .models import Book  # Импортируем модель
class BookForm(forms.ModelForm):
    class Meta:
        # Эта форма будет работать с моделью Book
        model = Book
        # Поля модели, которые должны отображаться в веб-форме;
        # Можно вывести только часть полей из модели.
        fields = ('name', 'isbn', 'pages')  # '__all__' или все.
		widgets = {
			'title': forms.TextInput(attrs={'class': 'form-control'}),
			'content': forms.Textarea(attrs={'class': 'form-control'}),
		}
```

#### 3. Объект формы (экземпляр, созданный на основе класса формы) передаётся в специальный view-класс.
```python
# library/views.py
from django.views.generic.edit import CreateView
from .forms import BookForm
class BookView(CreateView):  # Создаём свой класс, наследуем его от CreateView
    # C какой формой будет работать этот view-класс
    form_class = BookForm
    # Какой шаблон применить для отображения веб-формы
    template_name = 'library/new_book.html'
    # Куда переадресовать пользователя после того, как он отправит форму
    success_url = '/thankyou/'
```

#### 4. View-класс передаёт объект формы в шаблон, создаёт и возвращает пользователю страницу с веб-формой.
```django
<form method="post" action="{% url 'library:new_book' %}">
  {{ form.as_p }}
  <input type="submit" value="Отправить">
</form>
```

### Пример создания формы

```python
# models.py
from django.db import models
from django.core.validators import MinValueValidator

# Создадим модель, в которой будем хранить данные о книгах
class Book(models.Model):
    name = models.CharField(max_length=200)   # Название
    isbn = models.CharField(max_length=100)   # Индекс издания
    pages = models.IntegerField(              # Количество страниц
        validators=[MinValueValidator(1)]
    )
```

```python
# forms.py
from django import forms  # Импортируем модуль forms, из него возьмём класс ModelForm
from .models import Book  # Импортируем модель, чтобы связать с ней форму

class BookForm(forms.ModelForm):
    class Meta:
        # Эта форма будет работать с моделью Book
        model = Book

        # Здесь перечислим поля модели, которые должны отображаться в веб-форме;
        # при необходимости можно вывести в веб-форму только часть полей из модели.
        fields = ('name', 'isbn', 'pages')
```

### Метод save()
Метод `save()` в `ModelForm` в Django служит для сохранения данных формы в базу данных.
Если форма связана с моделью, то вызов `form.save()` создаст и сохранит объект модели.

```python
form = MyModelForm(request.POST)

if form.is_valid():
    # сохраняет форму и возвращает экземпляр объекта модели
    instance = form.save()
```
```python
instance = MyModel.objects.get(pk=1)
form = MyModelForm(request.POST, instance=instance)

if form.is_valid():
    form.save()
```
```python
form = MyModelForm(request.POST)

if form.is_valid():
    instance = form.save(commit=False)
    instance.some_field = 'Some Value'
    instance.save()
```

## class Meta
Используется для описания дополнительной информации о `ModelForm`

```python
class MyForm(forms.ModelForm):
	class Meta:
		<params>
```

### model
Модель, используемая для создания формы

```python
class Meta:
	model = MyModel
```

### fields
Список полей для включения в форму

```python
class Meta:
	model = MyModel
	fields = ['field1', 'field2']
```

### exclude
Список полей, которые следует исключить из формы

```python
class Meta:
	model = MyModel
	exclude = ['field3']
```

### widgets
Позволяет изменять виджеты формы по умолчанию для определенных полей

```python
class Meta:
	model = MyModel
	fields = ['field1', 'field2']
	widgets = {
		'field1': forms.Textarea(attrs={'cols': 80, 'rows': 20}),
	}
```

### field_classes
Словарь, сопоставляющий имя поля с Field

```python
class Meta:
	model = MyModel
	field_classes = {
		'field1': MyCustomFieldClass,
	}
```

### labels
Словарь, определяющий метки для полей формы

```python
class Meta:
	model = MyModel
	labels = {
		'field1': 'My custom label',
	}
```

### help_texts
Словарь, определяющий текст помощи для полей формы

```python
class Meta:
	model = MyModel
	help_texts = {
		'field1': 'Some useful help text.',
	}
```

### error_messages
Словарь, определяющий пользовательские сообщения об ошибках для полей формы

```python
class Meta:
	model = MyModel
	error_messages = {
		'field1': {
			'required': "This field is required",
		},
	}
```

### localized_fields
Список полей, которые должны быть локализованы.

### formfield_callback
Это вызываемый объект\
Принимает имя поля модели и возвращает экземпляр поля формы (или None, если стандартное поле формы подходит).

### ordering
Порядок полей в форме

```python
class Meta:
	model = MyModel
	fields = ['field1', 'field2', 'field3']
	ordering = ['field3', 'field1', 'field2']
```

### error_class
Класс для хранения ошибок валидации формы

### formfield_callback
Выываемый объект\
Принимает имя поля модели и возвращает экземпляр поля формы (или None, если стандартное поле формы подходит)

```python
def formfield_for_choice_field(db_field, request, kwargs):
	return db_field.formfield(kwargs)

class Meta:
	model = MyModel
	formfield_callback = formfield_for_choice_field
```

## csrf_token
Токен защиты от межсайтовых запросов

```jinja
{% csrf_token %}
```

## Validators

### after validation

Если все валидаторы формы вернули `True` — форма считается валидной, и метод формы `.is_valid()` вернёт `True`.
```python
# views.py
from django.shortcuts import redirect
def user_contact(request):
    # Проверяем, получен POST-запрос или какой-то другой:
    if request.method == 'POST':
    
        # Создаём объект формы класса ContactForm
        # и передаём в него полученные данные
        form = ContactForm(request.POST)
        
        # Если все данные формы валидны - работаем с "очищенными данными" формы
        if form.is_valid():
        
            # Берём валидированные данные формы из словаря form.cleaned_data
            name = form.cleaned_data['name']
            email = form.cleaned_data['email']
            subject = form.cleaned_data['subject']
            message = form.cleaned_data['body']
            
            # При необходимости обрабатываем данные
            # ...
            # сохраняем объект в базу
            form.save()
            
            # Функция redirect перенаправляет пользователя
            # на другую страницу сайта, чтобы защититься
            # от повторного заполнения формы
            return redirect('/thank-you/')
            
        # Если условие if form.is_valid() ложно и данные не прошли валидацию -
        # передадим полученный объект в шаблон,
        # чтобы показать пользователю информацию об ошибке
        # Заодно заполним все поля формы данными, прошедшими валидацию,
        # чтобы не заставлять пользователя вносить их повторно
        return render(request, 'contact.html', {'form': form})
        
    # Если пришёл не POST-запрос - создаём и передаём в шаблон пустую форму
    # пусть пользователь напишет что-нибудь
    form = ContactForm()
    return render(request, 'contact.html', {'form': form})
```


### function validator

Обычно пишутся в файле `validators.py`
> Затем для полей модели или формы указывают параметр `validators`, перечисляя в нём функции-валидаторы, которые должны проверить это поле.

#### Форма с моделью

```python
# validators.py

# Функция-валидатор:
def validate_not_empty(value):
    # Проверка "а заполнено ли поле?"
    if value == '':
        raise forms.ValidationError(
            'А кто поле будет заполнять, Пушкин?',
            params={'value': value},
        )
```
```python
# models.py
class Contact(models.Model):
    # К полю name подключаем валидатор, проверяющий, что поле не пустое.
    name = models.CharField(max_length=100, validators=[validate_not_empty])
    # К полю body тоже подключаем валидатор, проверяющий, что поле не пустое.
    body = models.TextField(validators=[validate_not_empty])
```

#### Форма без модели
```python
# validators.py

# Функция-валидатор:
def validate_not_empty(value):
    if value == '':
        raise forms.ValidationError(
            'А кто поле будет заполнять, Пушкин?',
            params={'value': value},
        )
```

```python
# forms.py

class ContactForm(forms.Form):
    name = forms.CharField(max_length=100, validators=[validate_not_empty])
    email = forms.EmailField()
    subject = forms.CharField(max_length=100)
    body = forms.CharField(widget=forms.Textarea)
    is_answered = forms.BooleanField(default=False)
```

### method validator

В классе формы для каждого отдельного поля можно создать метод `clean_<имя поля>`, он вызовется автоматически при создании объекта этого класса.
- Этот метод не получает никаких дополнительных параметров: он самостоятельно запросит значение из словаря `cleaned_data`.
- Значение, которое вернёт метод-валидатор, обновит значение соответствующего элемента словаря `cleaned_data`.

```python
# forms.py
from django import forms
from .models import Contact

class ContactForm(forms.ModelForm):

    class Meta:
        model = Contact
        fields = ('name', 'email', 'subject', 'body')

    # Метод-валидатор для поля subject
    def clean_subject(self):
        data = self.cleaned_data['subject']
        # Если пользователь не поблагодарил администратора - считаем это ошибкой
        if 'спасибо' not in data.lower():
            raise forms.ValidationError('Вы обязательно должны нас поблагодарить!')

        # Метод-валидатор обязательно должен вернуть очищенные данные,
        # даже если не изменил их
        return data
```

### Встроенные валидаторы моделей/форм
- Подключаются из `django.core.validators`
- Вызывают исключение ValidationError

```python
# Пример подключения валидатора
from django.core import validators

class News(models.Model):
	title = models.CharField(max_length=50,
		validators=[
			# Класс
			validators.RegexValidator
				regex='^.*',
				validation_func # Функция
			)
		]
	)

#### MinLengthValidator
Проверка на минимальную длину строки

```python
MinLengthValidator(<int>[, message=None])
# Без message выведется стандартное сообщение
```

#### MaxLengthValidator
Проверка на максимальную длину строки

```python
MaxLengthValidator(<int>[, message=None])
# Без message выведется стандартное сообщение
```

#### RegexValidator
Проверка на соответствие регулярному выражению

```python
RegexValidator(
	regexp=None# Cтрока или объект regexp
	[, message=None]       # Сообщение об ошибке
	[, code=None]          # Код ошибки def:'invalid'
	[, inverse_match=None] # if True value not regexp
	[, flags=0]# Флаги в виде строки
)
```

#### EmailValidator
Проверка Email на корректность

```python
EmailValidator(
	[, message=None]   # Сообщение об ошибке
	[, code=None]      # Код ошибки def:'invalid'
	[, whitelist=None] # Последовательность доменов без проверки
)
```

#### URLValidator
Проверка URL на корректность

```python
# Используется полем типа URLField
URLValidator(
	[, schemes=None] # def:['http', 'https', 'ftp', 'ftps']
	[, regex=None]   # Строка или объект regexp
	[, message=None] # Сообщение об ошибке
	[, code=None]    # Код ошибки def:'invalid'
)
```

#### ProhibitNullCharactersValidator
Не содержит ли строка нулевой символ \x00

```python
ProhibitNullCharactersValidator(
	[, message=None] # Сообщение об ошибке
	[, code=None]    # Код ошибки def:'invalid'
)
```

#### MinValueValidator
Проверка на минимальное число

```python
MinValueValidator(<int>[, message=None])
# Без message выведется стандартное сообщение
```

#### MaxValueValidator
Проверка на максимальное число

```python
MaxValueValidator(<int>[, message=None])
# Без message выведется стандартное сообщение
```

#### DecimalValidator
Проверка вещественного числа фиксированной точности\
Число представлено объектом decimal из модуля Decimal

```python
DecimalValidator(<MaxQtyDigit>, <FloatQtyDigit>)
```